# 안전 벡터맨

## 건설 현장 안전 사고방지 AI

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade jsonlines openai langchain langchain-openai langchain-community -q
!pip install chromadb==0.5.3 langchain-chroma tiktoken rank_bm25 -q
!pip install pymupdf pypdf pypdf2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.5/474.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.9 MB/s eta 0:00

In [5]:
import os

os.environ["OPENAI_API_KEY"] = 

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [6]:
from langchain.schema import Document
from glob import glob

path = '/content/drive/MyDrive/skala/건설현장지침/*.pdf' # './drive/MyDrive/8_papers/*.pdf'

glob(path)

['/content/drive/MyDrive/skala/건설현장지침/흙막이공사(C.I.P공법) 안전보건작업 지침.pdf',
 '/content/drive/MyDrive/skala/건설현장지침/건설기계 안전보건작업 지침.pdf',
 '/content/drive/MyDrive/skala/건설현장지침/건설공사 안전보건 설계 지침.pdf',
 '/content/drive/MyDrive/skala/건설현장지침/건설공사 돌관작업 안전보건작업 지침.pdf',
 '/content/drive/MyDrive/skala/건설현장지침/흙막이공사(띠장긴장공법, Prestressed Wale Method) 안전보건 작업지침.pdf',
 '/content/drive/MyDrive/skala/건설현장지침/흙막이공사(엄지말뚝 공법) 안전보건작업 지침.pdf',
 '/content/drive/MyDrive/skala/건설현장지침/흙막이공사(강널말뚝, Sheet Pile)의 안전보건작업지침.pdf',
 '/content/drive/MyDrive/skala/건설현장지침/흙막이공사(Soil Nailing 공법) 안전보건작업 지침.pdf',
 '/content/drive/MyDrive/skala/건설현장지침/흙막이공사 (SCW 공법) 안전보건작업지침.pdf',
 '/content/drive/MyD

In [7]:
import glob
import re
from langchain_community.document_loaders import PyPDFLoader  # PyPDFLoader 추가
from langchain.schema import Document

# PDF 파일 경로 설정
pdf_files = glob.glob('/content/drive/MyDrive/skala/건설현장지침/*.pdf')

# 전처리 함수 정의
def clean_text(text):
    # 페이지 번호 및 특수문자 제거
    text = re.sub(r'\n{2,}', ' ', text)  # 연속 개행을 공백으로 변환
    text = re.sub(r'[\n]', ' ', text)     # 일반 개행도 공백으로 변환
    text = re.sub(r'[^\S\n]+', ' ', text) # 다중 공백 제거
    text = text.strip()  # 양끝 공백 제거
    return text

# PDF 파일 로드 및 전처리
all_papers = []

for i, path_paper in enumerate(pdf_files):
    try:
        loader = PyPDFLoader(path_paper)  # PyPDFLoader 사용
        pages = loader.load()
    except:
        print(f"{path_paper} 로드 실패, 스킵합니다.")
        continue

    # PDF 페이지 병합 및 전처리
    doc = Document(page_content='', metadata={'index': i, 'source': pages[0].metadata['source']})
    for page in pages:
        clean_content = clean_text(page.page_content)
        doc.page_content += clean_content + " "

    all_papers.append(doc)

print(f"총 {len(all_papers)}개의 문서가 로드되었습니다.")
print("샘플 문서:", all_papers[:3])


총 12개의 문서가 로드되었습니다.
샘플 문서: [Document(metadata={'index': 0, 'source': '/content/drive/MyDrive/skala/건설현장지침/흙막이공사(C.I.P공법) 안전보건작업 지침.pdf'}, page_content='KOSHA GUIDE C - 63 - 2012 흙막이공사(C.I.P공법) 안전보건작업 지침 2012. 8 한 국 산 업 안 전 보 건 공 단 안전보건기술지침의 개요 ㅇ 제정자 : 김경순 ㅇ 제정경과 - 2009년 9월 건설안전분야 제정위원회 심의 - 2009년 11월 총괄제정위원회 심의 - 2012년 7월 건설안전분야 제정위원회 심의(개정) ㅇ 관련규격 및 자료 - 건축공사 표준시방서(국토해양부) - 가설공사 표준시방서(국토해양부) - KOSHA GUIDE C-39-2011(굴착공사 안전작업 지침) ㅇ 관련법규, 규칙, 고시 등 - 「산업안전보건기준에 관한 규칙」제2편 제4장 제2절 굴착작업 등의 위험방지 ㅇ 기술지침의 적용 및 문의 이 기술지침에 대한 의견 또는 문의는 한국산업안전보건공단 홈 페이지 안전보건기술지침 소관 분야별 문의처 안내를 참고하시기 바랍니다. 공표일자 : 2012년 8월 27일 제 정 자 : 한국산업안전보건공단 이사장 KOSHA GUIDE C - 63 - 2012 - 1 - 흙막이공사(C.I.P공법) 안전보건작업 지침 1. 목적 이 지침은「산업안전보건기준에 관한 규칙」(이하 “안전보건규칙”이라 한다) 제2편 제4장 제2절 굴착작업 등의 위험방지 규정에 따라, C.I.P 흙막이공사 작업과정에서의 안전보건작업 지침을 정함을 목적으로 한다. 2. 적용범위 이 지침은 건설공사 현장의 C.I.P 흙막이공사 작업에 적용한다. 3. 정의 (1) 이 지침에서 사용되는 용어의 뜻은 다음과 같다. (가) “C.I.P(Cast In Place Pile)”란 지반 천공장비를 사용, 소정의 심도까지 천공하여 토사를 배출시킨 후 공

In [8]:
import tiktoken

encoder = tiktoken.encoding_for_model('gpt-4o-mini') # 텍스트를 토큰으로 변환하기 위해, OpenAI 제공 패키지 tiktoken 사용
for paper in all_papers:
    print(len(encoder.encode(paper.page_content)), paper.metadata['source']) # 각 문서별 토큰 갯수 확인

5477 /content/drive/MyDrive/skala/건설현장지침/흙막이공사(C.I.P공법) 안전보건작업 지침.pdf
18346 /content/drive/MyDrive/skala/건설현장지침/건설기계 안전보건작업 지침.pdf
6201 /content/drive/MyDrive/skala/건설현장지침/건설공사 안전보건 설계 지침.pdf
4894 /content/drive/MyDrive/skala/건설현장지침/건설공사 돌관작업 안전보건작업 지침.pdf
8560 /content/drive/MyDrive/skala/건설현장지침/흙막이공사(띠장긴장공법, Prestressed Wale Method) 안전보건 작업지침.pdf
6501 /content/drive/MyDrive/skala/건설현장지침/흙막이공사(엄지말뚝 공법) 안전보건작업 지침.pdf
6492 /content/drive/MyDrive/skala/건설현장지침/흙막이공사(강널말뚝, Sheet Pile)의 안전보건작업지침.pdf
5642 /content/drive/MyDrive/skala/건설현장지침/흙막이공사(Soil Nailing 공법) 안전보건작업 지침.pdf
5011 /content/drive/MyDrive/skala/건설현장지침/흙막이공사 (SCW 공법) 안전보건작업지침.pdf
7085 /cont

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=800, # 800 토큰 단위 (GPT-4o-mini 기준)
    chunk_overlap=80,
)


token_chunks = token_splitter.split_documents(all_papers)
print(len(token_chunks))

125


In [10]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small') # 'text-embedding-3-large'

Chroma().delete_collection()
db = Chroma.from_documents(documents=token_chunks, # 이 코드에서 Chroma.from_document()이 실행되면 embedding을 사용하여 텍스트를 벡터로 변환하고 DB에 저장함.
                           embedding=embeddings,
                           #persist_directory="./chroma_Web", #  디스크에 저장하고 싶을 때 사용 #'./drive/MyDrive/8_papers./*.pdf'
                           collection_metadata={'hnsw:space':'l2'} # ChromaDB는 고차원 벡터 검색을 빠르게 수행하기 위해 HNSW 알고리즘을 사용, "hnsw:space"는 HNSW 검색 공간 거리 계산 방식 지정 키, "l2"는 L2 거리(유클리드 거리, Euclidean Distance) 의미
                           )
# Top 5 검색하기
retriever = db.as_retriever(search_kwargs={"k": 5}) # db에서 벡터 검색을 수행할 수 있도록 설정함, 데이터 저장이 완료된 이후에 retriever를 통해 검색 가능

# filter 옵션을 통해 특정 메타데이터를 가진 벡터만 검색 가능
# retriever = db.as_retriever(search_kwargs={"k": 5,"filter":{'author':'Sugnryel Lim'}})

retriever.invoke('안전')

[Document(id='1ed7d328-e537-42e7-99af-ff962e0cb7db', metadata={'index': 2, 'source': '/content/drive/MyDrive/skala/건설현장지침/건설공사 안전보건 설계 지침.pdf'}, page_content='안전 보건에 미치는 유해․위험 요인을 고려하여 설계 에 반영한다. (라) 구조물 시공을 위하여 설치되는 흙막이 지보공, 거푸집동바리, 비계 등 가설구조물에 대한 안전성 검토와 근로자의 안전 보건을 위한 안전시설이 설치될 수 있도록 설계도서에 반영한다. (마) 구조물 공사 중 동시에 발생하는 다른 공종간의 중첩 또는 간섭으로 인한 위험요인을 고려하여 시공순서를 설계도서에 반영한다. (바) 강구조물 설계 시 부재의 운반, 거치, 가조립 및 본조립 등 작업 절차상 위험요인을 고려하여 안전한 방법으로 작업이 이루어질 수 있도록 예방대책 을 설계도서에 반영한다. (사) 강구조물 부재의 운반, 거치, 가조립 작업용 러그(Lug), 볼트(Bolt) 등은 줄걸이 및 안전시설 설치 등을 고려하여 부재의 강성과 품질의 손상을 최소화하면서 줄걸이와 안전시설을 설치할 수 있도록 설계 시 반영한다. (아) 강구조물 부재의 가조립 단계에서 안전시설 및 작업대 등을 선행하여 설치하 는 작업방법을 설계에 반영하기 곤란한 경우 근로자 추락 방지를 위한 추락 및 낙하물방지망 설치 등 2차 안전대책을 고려하여 설계에 반영한다. 5.4 마감공사 (1) 마감공사 (가) 설계자는 마감공사 진행 단계에서 여러 공정이 동시에 진행됨에 따라 공종 또는 공정간 간섭 및 중첩되는 사항을 설계도서에 명시한다. (나) 마감공사 진행 단계에서 공종간 간섭이 발생하는 위치(또는 구간)의 시공 절차, 방법 등을 제시하여 시공단계의 위험요인을 최소화하도록 설계도서에 제시한다. KOSHA GUIDE C - 18 - 2015 - 10 - (다) 마감공사에 사용되는 도료, 유기용제, 방수제

In [11]:
# vector store에서 유사도 확인하기
query = "How does Exaone achieve good evaluation results?"
db.similarity_search_with_score(query) # 유사도 스코어는 0 ~ 1 사이 값을 가지며, 1에 가까울 수록 더 유사한 문서이다.

# 참고로, HNSW에서 l1은 코사인 유사도로 -1 ~ 1 사이 값을 가진다. 두 벡터가 동일한 방향을 가리킬 수록 1에 가까워지며 유사도가 높음
  # cos 0∘ = 1 → 완전히 같은 방향 (유사도가 1)
  # cos 90∘ = 0 → 서로 직교 (연관성이 없음)
  # cos 180∘ = −1 → 완전히 반대 방향 (유사도가 -1)

# l2는 유클리드 거리 유사도로, 0에 가까울 수록 유사도가 높다. (벡터들이 더 가까움)
  # 두 점(벡터) 사이의 직선 거리(즉, 피타고라스 정리 기반의 거리)를 측정하여 유사도를 판단하는 방식
  # 두 벡터의 차이의 제곱합을 계산하여 루트를 씌운 값이 유클리드 거리

[(Document(id='6ab764ab-8577-44c4-bf2a-d9e11de1b3f6', metadata={'index': 7, 'source': '/content/drive/MyDrive/skala/건설현장지침/흙막이공사(Soil Nailing 공법) 안전보건작업 지침.pdf'}, page_content='네일의 표준단면도 (2) 와이어메쉬 : 굴착표면에 뿜어붙이기 콘크리트(Shotcrete)의 부착력을 증진 시키기 위한 철망을 말한다. (3) 연결철근 : 네일에 작용되는 인발력을 인근의 네일로 하중분배를 시키기 위 한 철근을 말한다. KOSHA GUIDE C - 13 - 2012 - 2 - (4) 지압판 : 굴착면의 토사가 붕괴되려는 힘을 네일로 전달시키기 위한 부재를 말하며, 주로 150x150x9T의 강판을 사용하여 네일의 두부에 설치한다. <그림 2> 네일 설치 평면도 (5) 너트(Nut) : 지압판을 네일에 고정하여 지압판이 네일에서 이탈되려는 것을 방지하는 기능을 하는 부재를 말한다. (6) 스페이서(Spacer) : 네일이 천공 홀(hole) 단면의 중앙에 위치할 수 있도록 하는 부재로서 주로 1.5m 이내의 간격으로 설치하며, 네일이 주입재 (Grouting재)의 중앙에 위치하도록 하여 주입재와 지반과의 부착강도를 충 분히 발휘하게 하여야 한다. (7) 주입용 관 : 주입재를 주입하기 위한 관으로서 천공 홀 바닥에서부터 주입 될 수 있도록 설치한다. 4. 건설주체별 의무 4.1 설계자의 의무 (1) 대상 지반에 대한 지질조사, 가스관․통신선로․상수관․하수관․인근 구조 물의 기초 등 지하매설물 조사, 인근 구조물․고압전선로 등 지상 장애물 조사, 장비의 운행경로 등 현황조사를 실시하여야 한다. (2) 정착부의 지반에 대한 토질정수를 결정하기 위한 지질조사․토질시험 등을 실시하고 그 결과를 이용한 설계를 하여야 한다. (3) 정착부는 지하구조물 또는 지표면의 역방향 경사 등으

In [12]:
def retriever_with_score(query):
    docs, scores = zip(*db.similarity_search_with_score(query)) # * 연산자는 리스트의 각 요소를 개별적으로 풀어서(zip 해체) 따로 추출함, 여기서는 scores만 따로 추출
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [13]:
# Query 검색
# RunnableLambda : 함수를 Runnable로 Wrap

from langchain_core.runnables import RunnableLambda
unique_docs = RunnableLambda(
    retriever_with_score).invoke("How does Exaone achieve good evaluation results?")
# 함수에 직접 invoke를 실행 가능하도록 RunnableLamda()로 묶음
unique_docs

(Document(id='6ab764ab-8577-44c4-bf2a-d9e11de1b3f6', metadata={'index': 7, 'source': '/content/drive/MyDrive/skala/건설현장지침/흙막이공사(Soil Nailing 공법) 안전보건작업 지침.pdf', 'score': 1.516577124595642}, page_content='네일의 표준단면도 (2) 와이어메쉬 : 굴착표면에 뿜어붙이기 콘크리트(Shotcrete)의 부착력을 증진 시키기 위한 철망을 말한다. (3) 연결철근 : 네일에 작용되는 인발력을 인근의 네일로 하중분배를 시키기 위 한 철근을 말한다. KOSHA GUIDE C - 13 - 2012 - 2 - (4) 지압판 : 굴착면의 토사가 붕괴되려는 힘을 네일로 전달시키기 위한 부재를 말하며, 주로 150x150x9T의 강판을 사용하여 네일의 두부에 설치한다. <그림 2> 네일 설치 평면도 (5) 너트(Nut) : 지압판을 네일에 고정하여 지압판이 네일에서 이탈되려는 것을 방지하는 기능을 하는 부재를 말한다. (6) 스페이서(Spacer) : 네일이 천공 홀(hole) 단면의 중앙에 위치할 수 있도록 하는 부재로서 주로 1.5m 이내의 간격으로 설치하며, 네일이 주입재 (Grouting재)의 중앙에 위치하도록 하여 주입재와 지반과의 부착강도를 충 분히 발휘하게 하여야 한다. (7) 주입용 관 : 주입재를 주입하기 위한 관으로서 천공 홀 바닥에서부터 주입 될 수 있도록 설치한다. 4. 건설주체별 의무 4.1 설계자의 의무 (1) 대상 지반에 대한 지질조사, 가스관․통신선로․상수관․하수관․인근 구조 물의 기초 등 지하매설물 조사, 인근 구조물․고압전선로 등 지상 장애물 조사, 장비의 운행경로 등 현황조사를 실시하여야 한다. (2) 정착부의 지반에 대한 토질정수를 결정하기 위한 지질조사․토질시험 등을 실시하고 그 결과를 이용한 설계를 하여야 한다. (3) 정

In [14]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)


In [15]:
# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("user", '''당신은 건설 현장 안전 도우미 AI입니다. 다음의 Context를 이용하여 Question에 답변하세요.

### Chain-of-Thought : 안전 지침을 열고 해당 작업에 대한 준비 사항을 확인한다. 준비 사항 중 잊지 말아야 할 것을 기록한다.
시공 순서를 탐색한다. 예를 들어 지하 연속벽의 경우 안내벽 설치부터 콘크리트 타설까지 진행된다. 공정 순서를 우선 나열한 뒤, 각 공정에 대해 주요 안전사항을 기록한다.
안전화나 고도와 같은 기초 안전 내용보다는 압송 장치와 같이 디테일해서 놓칠 수 있는 부분을 찾아서 기록한다.
만일 작업의 특정 공정에 대해 물어본 경우, 내용을 구체적으로 기록한다.

### Condition :
1. 공정 순서를 먼저 나열한 후, 각 공정 단계별로 소제목을 구성하고, 단계별 중요도가 높은 사항을 기준으로 작성한다.
2. 어느 조항을 참조하는 지 작성하지 않는다.
3. 만약 모든 Context를 다 확인해도 정보가 없다면, 웹 검색을 통해 추가 정보를 검색한다.
4. 답변은 한국어로 작성한다.
5. 모든 시공 절차에 대한 내용을 작성한다.

### Output Example :
공정 순서:
1. 안내벽 설치 단계
2. 플랜트 설치 단계
3. 선행굴착 및 본굴착 단계

1. 안내벽 설치 단계
   a. 안내벽 설치 전, 지반 붕괴 위험이 있으면 흙막이지보공을 설치한다.
   b. 야간작업 시 충분한 조명(75럭스 이상)과 형광벨트, 경광등을 설치한다.
   c. 굴착장비가 사면에 지나치게 근접해 붕괴되지 않도록 한다.

2. 플랜트 설치 단계
   a. 구동벨트는 철망으로 감싸 끼임 사고를 방지한다.
   b. 사일로 점검을 위한 사다리 및 난간을 반드시 설치한다.
   c. 배관 연결 상태를 점검하고, 펌프 압력으로 유동되지 않도록 고정한다.

3. 선행굴착 및 본굴착 단계
   a. 트렌치 커터 이동 시 지반침하 방지를 위해 콘크리트를 타설하거나 철판을 깐다.
   b. 크레인 작업 구역에 근로자가 접근하지 않도록 통제한다.
   c. 굴착 중 지하 매설물(가스관, 상수도관 등) 위치를 사전에 확인하고 보호 조치를 취한다.
---
Context: {context}
---
Question: {question}''')
])

prompt.pretty_print()



================================ Human Message =================================

당신은 건설 현장 안전 도우미 AI입니다. 다음의 Context를 이용하여 Question에 답변하세요.

### Chain-of-Thought : 안전 지침을 열고 해당 작업에 대한 준비 사항을 확인한다. 준비 사항 중 잊지 말아야 할 것을 기록한다.
시공 순서를 탐색한다. 예를 들어 지하 연속벽의 경우 안내벽 설치부터 콘크리트 타설까지 진행된다. 공정 순서를 우선 나열한 뒤, 각 공정에 대해 주요 안전사항을 기록한다.
안전화나 고도와 같은 기초 안전 내용보다는 압송 장치와 같이 디테일해서 놓칠 수 있는 부분을 찾아서 기록한다.
만일 작업의 특정 공정에 대해 물어본 경우, 내용을 구체적으로 기록한다.

### Condition :
1. 공정 순서를 먼저 나열한 후, 각 공정 단계별로 소제목을 구성하고, 단계별 중요도가 높은 사항을 기준으로 작성한다.
2. 어느 조항을 참조하는 지 작성하지 않는다.
3. 만약 모든 Context를 다 확인해도 정보가 없다면, 웹 검색을 통해 추가 정보를 검색한다.
4. 답변은 한국어로 작성한다.
5. 모든 시공 절차에 대한 내용을 작성한다.

### Output Example :
공정 순서:
1. 안내벽 설치 단계
2. 플랜트 설치 단계
3. 선행굴착 및 본굴착 단계

1. 안내벽 설치 단계
   a. 안내벽 설치 전, 지반 붕괴 위험이 있으면 흙막이지보공을 설치한다.
   b. 야간작업 시 충분한 조명(75럭스 이상)과 형광벨트, 경광등을 설치한다.
   c. 굴착장비가 사면에 지나치게 근접해 붕괴되지 않도록 한다.

2. 플랜트 설치 단계
   a. 구동벨트는 철망으로 감싸 끼임 사고를 방지한다.
   b. 사일로 점검을 위한 사다리 및 난간을 반드시 설치한다.
   c. 배관 연결 상태를 점검하고, 펌프 압력으로 유동되지 않도록 고정한다.

3. 선행굴착 및 본굴착 단계
   

In [16]:
translate_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '주어진 질문을 영어로 변환하세요.'),
        ('user', 'Question: {question}')
    ]
)
translate_chain = translate_prompt | llm | StrOutputParser()

def format_docs(docs):
    return "\n---\n".join([doc.page_content+ '\nURL: '+ doc.metadata['source'] for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결

rag_chain = (
    {"context": translate_chain | retriever | format_docs, "question": RunnablePassthrough()}
    # context는 질문을 번역하고 검색한 문서를 텍스트로 변환한 것이며, question은 원래 질문을 그대로 유지
    # retriever : question을 받아서 context 검색: document 반환, format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    # context (검색된 문서)와 question (질문)을 이용하여 LLM에게 입력할 최종 프롬프트를 생성
    | llm
    | StrOutputParser()
)

In [17]:
questions = [
    '오늘  SCW 공법을 사용해서 흙막이 공사 할건데 안전 수칙 알려줘'
]
result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    ans = ans.replace('.','.\n')
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')


Question: 오늘  SCW 공법을 사용해서 흙막이 공사 할건데 안전 수칙 알려줘
Answer: 공정 순서:
1.
 안내벽 설치 단계
2.
 플랜트 설치 단계
3.
 천공 및 굴착 단계
4.
 시멘트 밀크 주입 단계
5.
 콘크리트 타설 단계

1.
 안내벽 설치 단계
   a.
 안내벽은 철근콘크리트나 H형 강재를 사용하여 설치하며, 정확한 위치와 수직도를 확보해야 한다.

   b.
 안내벽 설치 전, 지반의 안정성을 확인하고 무너짐의 우려가 있을 경우 양질의 토사로 치환하거나 안전구배를 확보해야 한다.

   c.
 안내벽과 장비 사이에 우수 등 지표수의 유입을 방지하기 위한 조치를 취해야 한다.


2.
 플랜트 설치 단계
   a.
 플랜트 설치 시 가설 전기설비에 대한 안전시설을 확보하고, 전기안전시설을 설치해야 한다.

   b.
 플랜트 주변에는 관계근로자 외 출입을 금지하고, 안전 표지판을 설치하여 안전사고를 예방해야 한다.

   c.
 시멘트 밀크의 공급 경로를 사전에 계획하고 점검하여 유출 사고를 방지해야 한다.


3.
 천공 및 굴착 단계
   a.
 천공 작업 시, 장비의 안전한 위치를 확보하고, 지반의 붕괴를 방지하기 위한 조치를 취해야 한다.

   b.
 굴착면의 높이가 2미터 이상일 경우, 추가적인 안전 조치를 마련해야 한다.

   c.
 굴착 중 지하 매설물의 위치를 사전에 확인하고, 보호 조치를 취해야 한다.


4.
 시멘트 밀크 주입 단계
   a.
 시멘트 밀크 주입 시, 주입 호스의 연결부 상태를 점검하여 파손으로 인한 인명피해를 예방해야 한다.

   b.
 주입 작업 중에는 관계근로자 외 출입을 금지하고, 안전한 작업 환경을 유지해야 한다.

   c.
 주입 작업 후에는 주변 환경에 미치는 영향을 최소화하기 위한 조치를 취해야 한다.


5.
 콘크리트 타설 단계
   a.
 콘크리트 타설 시, 타설 주변에 안전난간을 설치하여 추락사고를 예방해야 한다.

   b.
 타설 작업 중에는 신호수를 배치하여 협

In [ ]:
import gradio as gr
import asyncio

def analyze_excavation_safety(question):
    """
    질문을 받아 RAG 모델을 호출하여 안전 정보를 제공하는 함수
    """
    result = rag_chain.invoke(question)  # RAG 체인을 활용한 응답 생성
    formatted_result = result.replace('.', '.\n')  # 가독성 향상
    return formatted_result

# ✅ Gradio UI 구성
with gr.Blocks(css=".gradio-container {max-width: 800px; margin: auto; font-family: 'Arial', sans-serif;}") as demo:
    gr.Markdown("## 🏗️ 건설현장 안전사고 방지 AI")
    gr.Markdown("건설현장에서 발생할 수 있는 사고에 대해 질문하면, 벡터 DB와 RAG 기술을 활용한 학습 데이터 기반의 정보를 제공합니다.")

    with gr.Row():
        with gr.Column():
            question_input = gr.Textbox(
                label="💬 건설현장 관련 사고 질문 입력",
                placeholder="예: 오늘 SCW 공법을 사용할 건데 안전 수칙 알려줘",
                lines=2,
                interactive=True
            )
            submit_button = gr.Button("🚀 분석 실행")

        with gr.Column():
            output_box = gr.Textbox(
                label="📝 분석 결과",
                placeholder="AI가 분석한 결과가 여기에 표시됩니다.",
                lines=10,
                interactive=False
            )

    submit_button.click(fn=analyze_excavation_safety, inputs=[question_input], outputs=output_box)

    gr.Markdown("---")
    gr.Markdown("&copy; 2025 안전 벡터맨 All rights reserved")

# 실행
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2729a6243767fc75c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 83.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu1